<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/agents_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Preparando el cuaderno**
---
bla, bla, bla

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.
%pip install langgraph -qU  # Instalar la librería de grafos de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# # Importamos las clases necesarias para trabajar con cadenas
# from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI  # Esta tambien para DeepSeek
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

In [ ]:
from typing import Dict, List, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langgraph.graph import END, StateGraph


# 1. Actualizar el estado
class TranslationState(TypedDict):
    original_text: str
    source_language: str
    target_language: str
    translator_output: str
    accuracy_suggestions: List[str]  # Cambiado a específico para Revisor 1
    fluency_suggestions: List[str]   # Nuevo campo para Revisor 2
    editor_output: str
    translation_iterations: int

# 2. Modificar las plantillas
translator_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a senior lawyer-translator working for an international organisation on Intellectual Property. "
        "You have more than 10 years of professional experience, specializing in translation from {source_language} to {target_language}. "
        "With many years of experience in specialized translation, you use the most advanced translation techniques to provide high-quality, fluent, and accurate translations."
        "The translation should be ready for publication, and you need to ensure that a mistranslation may result in serious economic and legal consequences for your client."
        "Please ensure that the translation is perfect."
    )),
    ("human", "Your task is to provide a professional, high-quality translation from {source_language} to {target_language} of the following text:\n{original_text}."
    "The resulting translation should have the highest standards, and should convey all the meaning of the source text in a way that is fluent and appropriate for the target culture. "
])

accuracy_reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an expert Accuracy Reviewer, specializing in reviewing translations from {source_language} to {target_language}. Focus ONLY on: "
        "- Accuracy (by correcting errors of addition, mistranslation, omission or untranslated text.\n"
        "- Terminology (ensuring that the terminology used is appropriate for the context, that the use is coherent and consistent throughout the same document, etc.)\n"
        "- Other accuracy errors (proper noun translations, numerical consistency, technical accuracy.\n"
        "- Only suggest improvements that enhance the accuracy of the translation. Do not make any preferential changes.\n"
        "Return EXACTLY 'NO_SUGGESTIONS' if the accuracy of the translation is perfect and you think that no Accuracy improvements could be done.  "
        "Format: bullet points with 'ERROR: [issue] → SUGGESTION: [fix]'"
    )),
    ("human", (
        "Source ({source_language}):\n{original_text}\n\n"
        "Translation ({target_language}):\n{translator_output}\n\n"
        "Critical accuracy review (ONLY technical/terminology issues):"
    ))
])

fluency_reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an expert Fluency Reviewer, specializing in reviewing translations from {source_language} to {target_language}. Focus ONLY on:\n"
        "- Fluency (by applying {target_language} grammar, spelling and punctuation rules, and ensuring there are no unnecessary repetitions).\n"
        "- Style (by ensuring the translations reflect the style of the source text and takes into account any cultural context).\n"
        "- Other fluency errors (natural flow, idiomatic expressions, readability, cultural adaptation).\n"
        "- Only suggest improvements that enhance the fluency of the translation. Do not make any preferential changes.\n"
        "Return EXACTLY 'NO_SUGGESTIONS' if the fluency of the translation is perfect and you think that no Fluency improvements could be done."
        "Format: bullet points with 'ISSUE: [description] → IMPROVEMENT: [suggestion]'"
    )),
    ("human", (
        "Translation to refine:\n{translator_output}\n\n"
        "Target language: {target_language}\n"
        "Fluency and style improvements:"
    ))
])

editor_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a senior editor, having worked for more than 10 years as a lawyer-linguist in an Intellectual Property organisation."
        "You specialize in editing translations from {source_language} to {target_language}. "
        "Your task is to carefully read, and then edit, the translation by the Translator, and edit it"
        "by taking into account a list of expert suggestions and constructive criticisms by the Reviewers."
        "Synthesize inputs from both reviewers. Prioritize accuracy changes, then implement fluency improvements."
    )),
    ("human", (
        "Original Text ({source_language}):\n{original_text}\n\n"
        "Draft Translation:\n{translator_output}\n\n"
        "Accuracy Issues:\n{accuracy_suggestions}\n\n"
        "Fluency Suggestions:\n{fluency_suggestions}\n\n"
        "Produce a final, polished translation that is ready for publication and has the highest-quality standards:"
    ))
])

# 3. Configurar modelos (ajustar según necesidades)

# Con OpenAI

translator_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.3)
accuracy_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.1)  # Modelo más conservador
fluency_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)   # Modelo más creativo
editor_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.4)

# Con Deepseek
#model=deepseek-reasoner para R1
# translator_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.3)
# accuracy_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.1)  # Modelo más conservador
# fluency_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.7)  # Modelo más creativo
# editor_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.4)

# 4. Crear nuevas cadenas
translator_chain = translator_prompt | translator_llm
accuracy_review_chain = accuracy_reviewer_prompt | accuracy_llm
fluency_review_chain = fluency_reviewer_prompt | fluency_llm
editor_chain = editor_prompt | editor_llm

# 5. Actualizar funciones de nodo
def translate(state: TranslationState) -> dict:
    print("\n--- TRANSLATING ---")
    result = translator_chain.invoke(state)
    display(Markdown("📄 Traducción generada:\n" + result.content))
    return {"translator_output": result.content}

def review_accuracy(state: TranslationState) -> dict:
    print("\n--- ACCURACY REVIEW ---")
    result = accuracy_review_chain.invoke(state)
    content = result.content.strip()

    suggestions = []
    if content != "NO_SUGGESTIONS":
        suggestions = [line.strip() for line in content.split("\n") if line.strip()]
        print("⚠️ Sugerencias de precisión:")
        for sug in suggestions:
            display(Markdown(f"    {sug}"))
    else:
        print("✅ Sin problemas de precisión")

    return {"accuracy_suggestions": suggestions}

def review_fluency(state: TranslationState) -> dict:
    print("\n--- FLUENCY REVIEW ---")
    result = fluency_review_chain.invoke(state)
    content = result.content.strip()

    suggestions = []
    if content != "NO_SUGGESTIONS":
        suggestions = [line.strip() for line in content.split("\n") if line.strip()]
        print("✏️ Sugerencias de estilo:")
        for sug in suggestions:
            display(Markdown(f"    {sug}"))
    else:
        print("✅ Sin problemas de fluidez")

    return {"fluency_suggestions": suggestions}

def edit(state: TranslationState) -> dict:
    print("\n--- FINAL EDITING ---")
    result = editor_chain.invoke(state)
    print("🔖 Versión final:")
    display(Markdown(result.content))
    return {"editor_output": result.content}

# 6. Reconstruir el grafo
builder = StateGraph(TranslationState)

builder.add_node("translate", translate)
builder.add_node("review_accuracy", review_accuracy)
builder.add_node("review_fluency", review_fluency)
builder.add_node("edit", edit)

builder.set_entry_point("translate")

# Flujo principal CORREGIDO
builder.add_edge("translate", "review_accuracy")
builder.add_edge("review_accuracy", "review_fluency")
builder.add_edge("review_fluency", "edit")
builder.add_edge("edit", END)

graph = builder.compile()

# 8. Ejecutar
inputs = {
    "original_text": "Los Angeles firefighters were making modest progress in taming the region’s two largest fires on Saturday as they raced to suppress them ahead of high winds that were expected to intensify later in the day. After a night of expanded evacuation orders and spreading flames that continued to plunge the area into what Lindsey Horvath, a Los Angeles County supervisor, called 'unimaginable terror and heartbreak,' crews had contained 11 percent of the 22,660-acre Palisades fire and 15 percent of the 14,000-acre Eaton fire, near Altadena and Pasadena, according to Cal Fire.",
    "source_language": "inglés",
    "target_language": "español",
    "translation_iterations": 0
}
result = graph.invoke(inputs)


# 9. Con display(Markdown())
display(Markdown(f"""
### 📝 **Tracción Finalizada**
🗣️ **Origen:** {result['source_language']} → **Destino:** {result['target_language']}

🔤 **Texto original:**
*{result['original_text']}...*

🎯 **Traducción:**
{result.get('editor_output', '')}
"""))


--- TRANSLATING ---


📄 Traducción generada:
Los bomberos de Los Ángeles estaban logrando un progreso modesto en controlar los dos incendios más grandes de la región el sábado, mientras se apresuraban a sofocarlos antes de que los vientos fuertes, que se esperaba que se intensificaran más tarde en el día, llegaran. Después de una noche de órdenes de evacuación ampliadas y llamas en expansión que continuaron sumergiendo el área en lo que Lindsey Horvath, una supervisora del condado de Los Ángeles, describió como 'un terror y un dolor inimaginables', las brigadas habían contenido el 11 por ciento del incendio de Palisades, que abarca 22,660 acres, y el 15 por ciento del incendio de Eaton, que cubre 14,000 acres, cerca de Altadena y Pasadena, según Cal Fire.


--- ACCURACY REVIEW ---
⚠️ Sugerencias de precisión:


    - ERROR: "incendio de Palisades" → SUGGESTION: "incendio Palisades"

    - ERROR: "incendio de Eaton" → SUGGESTION: "incendio Eaton"

...

    Final refined version:

    "Los bomberos de Los Ángeles lograban un progreso modesto en controlar los dos incendios más grandes de la región el sábado, mientras se apresuraban a sofocarlos antes de que llegaran los fuertes vientos, que se esperaba se intensificarían más tarde en el día. Tras una noche de órdenes de evacuación ampliadas y llamas en expansión que continuaban sumergiendo el área en lo que Lindsey Horvath, una supervisora del condado de Los Ángeles, describió como 'un terror y un sufrimiento inimaginables', las brigadas habían contenido el 11 % del incendio de Palisades, que afecta 22,660 acres, y el 15 % del incendio de Eaton, que afecta 14,000 acres, cerca de Altadena y Pasadena, según informó Cal Fire."


--- FINAL EDITING ---
🔖 Versión final:


Los bomberos de Los Ángeles lograban un progreso modesto en controlar los dos incendios más grandes de la región el sábado, mientras se apresuraban a sofocarlos antes de que llegaran los fuertes vientos, que se esperaba se intensificarían más tarde en el día. Tras una noche de órdenes de evacuación ampliadas y llamas en expansión que continuaban sumergiendo el área en lo que Lindsey Horvath, una supervisora del condado de Los Ángeles, describió como 'un terror y un sufrimiento inimaginables', las brigadas habían contenido el 11 % del incendio Palisades, que afecta 22,660 acres, y el 15 % del incendio Eaton, que afecta 14,000 acres, cerca de Altadena y Pasadena, según informó Cal Fire.


### 📝 **Tracción Finalizada**  
🗣️ **Origen:** inglés → **Destino:** español

🔤 **Texto original:**  
*Los Angeles firefighters were making modest progress in taming the region’s two largest fires on Saturday as they raced to suppress them ahead of high winds that were expected to intensify later in the day. After a night of expanded evacuation orders and spreading flames that continued to plunge the area into what Lindsey Horvath, a Los Angeles County supervisor, called 'unimaginable terror and heartbreak,' crews had contained 11 percent of the 22,660-acre Palisades fire and 15 percent of the 14,000-acre Eaton fire, near Altadena and Pasadena, according to Cal Fire....*  

🎯 **Traducción:**  
Los bomberos de Los Ángeles lograban un progreso modesto en controlar los dos incendios más grandes de la región el sábado, mientras se apresuraban a sofocarlos antes de que llegaran los fuertes vientos, que se esperaba se intensificarían más tarde en el día. Tras una noche de órdenes de evacuación ampliadas y llamas en expansión que continuaban sumergiendo el área en lo que Lindsey Horvath, una supervisora del condado de Los Ángeles, describió como 'un terror y un sufrimiento inimaginables', las brigadas habían contenido el 11 % del incendio Palisades, que afecta 22,660 acres, y el 15 % del incendio Eaton, que afecta 14,000 acres, cerca de Altadena y Pasadena, según informó Cal Fire.
